# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [1]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [2]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [3]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-10-25'
start_date = '2016-01-01'
# Descargar el dataframe
arg_df = yf.download('ARS=X',start_date, today).reset_index()

arg_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2030,2023-10-18,$349.92,$350.05,$349.62,$349.92,$349.92,0
2031,2023-10-19,$350.07,$350.11,$349.85,$350.07,$350.07,0
2032,2023-10-20,$349.94,$350.08,$349.87,$349.94,$349.94,0
2033,2023-10-23,$349.96,$350.33,$349.61,$349.96,$349.96,0
2034,2023-10-24,$350.01,$350.07,$349.79,$350.01,$350.01,0


In [4]:
# TAREA: Validamos que no hay datos vacíos
arg_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [5]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = arg_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
new_names = {
    "Date": "ds",
    "Open": "y",
}

# Los renombramos con los nuevos nombres
df.rename(columns=new_names, inplace=True)
df['ds'] = df['ds'].dt.tz_localize(None)

In [6]:
df.tail()

,ds,y
2030,2023-10-18,$349.92
2031,2023-10-19,$350.07
2032,2023-10-20,$349.94
2033,2023-10-23,$349.96
2034,2023-10-24,$350.01


In [7]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de inflación en el peso argentino",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [8]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet()

In [9]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ehezyl6/bwa7z5le.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp4ehezyl6/it2mbvzb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=28486', 'data', 'file=/tmp/tmp4ehezyl6/bwa7z5le.json', 'init=/tmp/tmp4ehezyl6/it2mbvzb.json', 'output', 'file=/tmp/tmp4ehezyl6/prophet_model5ixx7mab/prophet_model-20231025174605.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:46:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:46:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [19]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=5000)
future.tail()

,ds
7030,2037-06-28
7031,2037-06-29
7032,2037-06-30
7033,2037-07-01
7034,2037-07-02


In [20]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

#future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-04
2,2016-01-05
3,2016-01-06
4,2016-01-07
...,...
7030,2037-06-28
7031,2037-06-29
7032,2037-06-30
7033,2037-07-01


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [21]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [22]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

#Muestra la tabla de la predicción de la inflación

,ds,yhat,yhat_lower,yhat_upper
7030,2037-06-28,"$2,263.41","$1,314.07","$3,252.39"
7031,2037-06-29,"$2,264.59","$1,307.78","$3,251.77"
7032,2037-06-30,"$2,265.32","$1,316.99","$3,256.32"
7033,2037-07-01,"$2,265.68","$1,322.38","$3,253.19"
7034,2037-07-02,"$2,266.12","$1,318.18","$3,250.33"


In [23]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

313.6340867339529

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [24]:
plot_plotly(m, forecast)

#Crea la gráfica con la predicción

In [25]:
plot_components_plotly(m, forecast)